In [1]:
pip install scikit-learn==0.21.2

     |████████████████████████████████| 6.7MB 2.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
# Vanishing/Exploding gradients problem

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [5]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [6]:
model.layers[1].updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

In [7]:
# Using batch normalization before the activation function layers

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [8]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
# Reusing pretrained layers

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]


In [10]:
X_train_A.shape

(43986, 28, 28)

In [11]:
X_train_B.shape


(200, 28, 28)

In [12]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [13]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [14]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [15]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [16]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5555 - accuracy: 0.8214 - val_loss: 0.3772 - val_accuracy: 0.8697
Epoch 2/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3460 - accuracy: 0.8811 - val_loss: 0.3304 - val_accuracy: 0.8852
Epoch 3/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3119 - accuracy: 0.8922 - val_loss: 0.3089 - val_accuracy: 0.8931
Epoch 4/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2931 - accuracy: 0.8985 - val_loss: 0.2908 - val_accuracy: 0.8984
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2808 - accuracy: 0.9025 - val_loss: 0.2797 - val_accuracy: 0.9001
Epoch 6/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2713 - accuracy: 0.9061 - val_loss: 0.2746 - val_accuracy: 0.9058
Epoch 7/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2637 - accuracy: 0.9093 - val_loss: 0.2680 - val_accuracy:

In [17]:
model_A.save("my_model_A.h5")



In [18]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [19]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [20]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 28ms/step - loss: 0.6698 - accuracy: 0.6150 - val_loss: 0.4986 - val_accuracy: 0.7921
Epoch 2/20
7/7 [==============================] - 0s 16ms/step - loss: 0.4683 - accuracy: 0.8100 - val_loss: 0.3956 - val_accuracy: 0.8844
Epoch 3/20
7/7 [==============================] - 0s 14ms/step - loss: 0.3731 - accuracy: 0.8950 - val_loss: 0.3261 - val_accuracy: 0.9168
Epoch 4/20
7/7 [==============================] - 0s 14ms/step - loss: 0.3056 - accuracy: 0.9250 - val_loss: 0.2791 - val_accuracy: 0.9391
Epoch 5/20
7/7 [==============================] - 0s 16ms/step - loss: 0.2605 - accuracy: 0.9600 - val_loss: 0.2449 - val_accuracy: 0.9625
Epoch 6/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2273 - accuracy: 0.9750 - val_loss: 0.2188 - val_accuracy: 0.9736
Epoch 7/20
7/7 [==============================] - 0s 14ms/step - loss: 0.2017 - accuracy: 0.9750 - val_loss: 0.1970 - val_accuracy: 0.9757
Epoch 8/20
7/7 [===========

In [21]:
model_A.layers

In [22]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [23]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [24]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
metrics=["accuracy"])

In [25]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 28ms/step - loss: 1.4115 - accuracy: 0.1900 - val_loss: 0.7492 - val_accuracy: 0.4990
Epoch 2/4
7/7 [==============================] - 0s 13ms/step - loss: 0.5293 - accuracy: 0.7400 - val_loss: 0.3556 - val_accuracy: 0.9249
Epoch 3/4
7/7 [==============================] - 0s 13ms/step - loss: 0.2758 - accuracy: 0.9500 - val_loss: 0.2294 - val_accuracy: 0.9726
Epoch 4/4
7/7 [==============================] - 0s 15ms/step - loss: 0.1857 - accuracy: 0.9850 - val_loss: 0.1703 - val_accuracy: 0.9817
Epoch 1/16
7/7 [==============================] - 0s 27ms/step - loss: 0.1522 - accuracy: 0.9950 - val_loss: 0.1670 - val_accuracy: 0.9828
Epoch 2/16
7/7 [==============================] - 0s 14ms/step - loss: 0.1491 - accuracy: 0.9950 - val_loss: 0.1642 - val_accuracy: 0.9828
Epoch 3/16
7/7 [==============================] - 0s 15ms/step - loss: 0.1464 - accuracy: 0.9950 - val_loss: 0.1616 - val_accuracy: 0.9828
Epoch 4/16
7/7 [===============

In [26]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0887 - accuracy: 0.9910


[0.088662289083004, 0.9909999966621399]

In [27]:
model_B_on_A.evaluate(X_test_B, y_test_B)



63/63 [==============================] - 0s 2ms/step - loss: 0.1324 - accuracy: 0.9895


[0.1324496865272522, 0.9894999861717224]

In [28]:
# Faster Optmizers

In [29]:
# Avoiding overfitting through regularization

In [30]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [31]:
# Dropout

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5675 - accuracy: 0.8031 - val_loss: 0.3707 - val_accuracy: 0.8622
Epoch 2/2
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4226 - accuracy: 0.8453 - val_loss: 0.3472 - val_accuracy: 0.8716


In [32]:
# Alpha Dropout

tf.random.set_seed(42)
np.random.seed(42)

In [33]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))


Epoch 1/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6654 - accuracy: 0.7594 - val_loss: 0.5934 - val_accuracy: 0.8412
Epoch 2/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5605 - accuracy: 0.7939 - val_loss: 0.5519 - val_accuracy: 0.8396
Epoch 3/20
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5283 - accuracy: 0.8051 - val_loss: 0.4866 - val_accuracy: 0.8590
Epoch 4/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5072 - accuracy: 0.8117 - val_loss: 0.4633 - val_accuracy: 0.8592
Epoch 5/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4922 - accuracy: 0.8187 - val_loss: 0.4641 - val_accuracy: 0.8586
Epoch 6/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4865 - accuracy: 0.8195 - val_loss: 0.4724 - val_accuracy: 0.8596
Epoch 7/20
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4727 - accuracy: 0.8240 - val_loss: 0.4590 - val_accuracy:

In [34]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4467 - accuracy: 0.8648


[0.44666436314582825, 0.864799976348877]

In [35]:
model.evaluate(X_train_scaled, y_train)


1719/1719 [==============================] - 3s 2ms/step - loss: 0.3318 - accuracy: 0.8873


[0.33179745078086853, 0.8873090744018555]

In [36]:
# MC Dropout

tf.random.set_seed(42)
np.random.seed(42)


In [37]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)


In [38]:
y_probas.shape

(100, 10000, 10)

In [39]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.01, 0.  , 0.97]],
      dtype=float32)

In [40]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.  , 0.18, 0.  , 0.52]],
      dtype=float32)

In [41]:
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.27, 0.  , 0.16, 0.  , 0.29]],
      dtype=float32)

In [42]:
y_pred = np.argmax(y_proba, axis=1)

In [43]:
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8646